# 2.3.2 Updatable Featurization Jobs and Datasets (tg.common.datasets.featurization.updatable)

## Overview

If the data in the dataset are changing, we may want to update dataset from time to time. However, we don't want to reprocess all the records, as it takes too much time. This problem is addressed by the `UpdatableDataset` class and jobs that create and process it.

The dataset consists of _revisions_. Each revision is essentially a dataframe, that may be partitioned into smaller dataframes for optimization reasons (as done in `FeaturizationJob`). Rows of each revision describes the entities (like passengers, customers, or articles), and the dataframe index must be a unique identifier for the entity. If several rows are available in different revisions for the same entity, the later takes precedence. 

When stored in S3 or other external storages, several datasets may be stored together the same way as we have seen in the `FeaturizationJob`: the data are the same, but processed by different featurizers. 

Revisions may be _major_ and _minor_. Major revision corresponds to reevauation of all entities. It **does not** physically remove data from the dataset, but effectively voids them - data older that latest major revision are not read by default. The minor revisions are supposed to be small updates that rolls on top of the major revision. Due to this design:

* it is possible to restore the dataset's state to any given time in the past. 
* it is also possible not to carry around the very old data all the time, as they can be voided by a major revision.

The structure of the files on the S3 or other storages is:

```
root
↳ revision_1
  ↳ featurizer_X
    ↳ partition_1_X_i.parquet
    ↳ partition_1_X_ii.parquet
  ↳ featurizer_Y
    ↳ partition_1_Y_i.parquet
    ↳ partition_1_Y_ii.parquet
↳ revision_2
  ↳ featurizer_X
    ↳ partition_2_X_i.parquet
    ↳ partition_2_X_ii.parquet
  ↳ featurizer_Y
    ↳ partition_2_Y_i.parquet
    ↳ partition_2_Y_ii.parquet
↳ description.parquet
```

`revision_*` folders and `partition_*` files are  usually GUIDs, `featurizer_*` have meaningful names. 

`description.parquet` is the file describing the revisions. For each revision we know:

* the timestamp it was produced at
* if the revision was major or minor
* and the version of the job that has produced the revision. If the version was updated, the major revision should be triggered.

Also, `description.parquet` plays a role of transactions keeper: all the jobs producing datasets upload this file at the very end. Since this file keeps record of others, incomplete upload will not have any effect (aside from consuming space on the remote storage).

When working with this dataset, we specify the time `T` and download revisions between `T` and the last major revision before `T`. We also specify one featurizer, and download data for that. The resulting local file structure is:

```
root
↳ revision_1
  ↳ partition_1_X_i.parquet
  ↳ partition_1_X_ii.parquet
↳ revision_2
  ↳ partition_2_X_i.parquet
  ↳ partition_2_X_ii.parquet
↳ description.parquet
```

## Creating and reading updatable datasets

To demonstrate the file structure above, we will create an artificial dataset witout any particular meaning. The simplest way to do it is using a method in `UpdatableDataset` class. As before, we will use `MemoryFileSyncer` to demonstrate the file structure on the remote host.

In [1]:
from tg.common.datasets.featurization import UpdatableDataset
from tg.common import MemoryFileSyncer
from datetime import datetime
import pandas as pd

mem = MemoryFileSyncer()

def time(day):
    return datetime(2020,1,1+day)

def create_dataframe(day, featurizer):
    N=5
    return pd.DataFrame(dict(
        day = [day] * N,
        featurizer = [featurizer] * N,
        id = list(range(day, day+N))
    )).set_index("id")

for day in [0,2,4,6]:
    record = UpdatableDataset.DescriptionItem(
        name = f'revision_{day}',
        timestamp = time(day),
        is_major = day%4==0,
        version = '')
    data = {featurizer: create_dataframe(day, featurizer) 
            for featurizer in ['featurizer_A','featurizer_B']}
    UpdatableDataset.write_to_updatable_dataset(
        syncer = mem,
        record = record,
        data = data
    )

Here is the list of files and folders, created in the `MemoryFileSyncer`:

In [2]:
list(mem.cache)

['revision_0/featurizer_B/data.parquet',
 'revision_0/featurizer_A/data.parquet',
 'description.parquet',
 'revision_2/featurizer_B/data.parquet',
 'revision_2/featurizer_A/data.parquet',
 'revision_4/featurizer_B/data.parquet',
 'revision_4/featurizer_A/data.parquet',
 'revision_6/featurizer_B/data.parquet',
 'revision_6/featurizer_A/data.parquet']

The description of the dataset:

In [3]:
pd.read_parquet(mem.get_file_stream('description.parquet'))

,name,timestamp,is_major,version
0,revision_0,2020-01-01,True,
1,revision_2,2020-01-03,False,
2,revision_4,2020-01-05,True,
3,revision_6,2020-01-07,False,


Here is how one revision looks like:

In [4]:
pd.read_parquet(mem.get_file_stream('revision_0/featurizer_A/data.parquet'))

,day,featurizer
id,,
0,0,featurizer_A
1,0,featurizer_A
2,0,featurizer_A
3,0,featurizer_A
4,0,featurizer_A


Here is the table, describing our dataset for `featurizer_A`: in columns, there are revisions, in rows, entities, and in cells, the `day` field of the dataframe. 

In [5]:
dfs = []
for day in [0,2,4,6]:
    df = pd.read_parquet(mem.get_file_stream(f"revision_{day}/featurizer_A/data.parquet"))
    dfs.append(df[['day']].rename(columns={'day':f'day_{day}'}))
summary_df = pd.concat(dfs,axis=1)
summary_df

,day_0,day_2,day_4,day_6
id,,,,
0,0.0,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN
2,0.0,2.0,NaN,NaN
3,0.0,2.0,NaN,NaN
4,0.0,2.0,4.0,NaN
5,NaN,2.0,4.0,NaN
6,NaN,2.0,4.0,6.0
7,NaN,NaN,4.0,6.0
8,NaN,NaN,4.0,6.0


Use this table for reference: we will now read this dataset at specific time periods, so you will be able to see which records are available, and at which time.

Now, let's read the dataset with the `UpdatableDataset` class. It will "download" the data to the local drive (in this particular example, it downloads from memory).

In [6]:
dataset = UpdatableDataset(
    location = './temp/updatable_dataset',
    featurizer_name = 'featurizer_A',
    syncer = mem
)

dataset.read(cache_mode='remake')

,day,featurizer
id,,
6,6,featurizer_A
7,6,featurizer_A
8,6,featurizer_A
9,6,featurizer_A
10,6,featurizer_A
4,4,featurizer_A
5,4,featurizer_A


**Note** the `download` argument. In the normal setup, we download dataset to the appropriate timeframe once with `download` method, and then just read from it. In this paragraph however we want to demonstrate downloading from different timeframes, so we will have to download with each read.

This is the "current" state of the dataset: 
* records 9-10 were added in the last minor revision (6)
* records 6-8 were updated in the last minor revision (6)
* records 4-5 were inherited from the revision (4)
* records 1-3 are lost, because the revision (4) is major

This is the list of local files:

In [7]:
from yo_fluq_ds import *

Query.folder('./temp/updatable_dataset','**/*').foreach(print)

temp/updatable_dataset/revision_4
temp/updatable_dataset/description.parquet
temp/updatable_dataset/revision_6
temp/updatable_dataset/revision_4/featurizer_A
temp/updatable_dataset/revision_4/featurizer_A/data.parquet
temp/updatable_dataset/revision_6/featurizer_A
temp/updatable_dataset/revision_6/featurizer_A/data.parquet


As we can see, indeed `UpdatableDataset` has only downloaded what is needed for the provided timeframe.

`UpdatableDataset` can restore it's state to any given point in the past:

In [8]:
dataset.read(to_timestamp=time(1), cache_mode='remake')

,day,featurizer
id,,
0,0,featurizer_A
1,0,featurizer_A
2,0,featurizer_A
3,0,featurizer_A
4,0,featurizer_A


As we see, "lost" records 1-3 are available.

In [9]:
dataset.read(to_timestamp=time(3), cache_mode='remake')

,day,featurizer
id,,
2,2,featurizer_A
3,2,featurizer_A
4,2,featurizer_A
5,2,featurizer_A
6,2,featurizer_A
0,0,featurizer_A
1,0,featurizer_A


Also, `UpdatableDataset` can provide you with changes, made between a given timestamps:

In [10]:
dataset.read(from_timestamp=time(1), to_timestamp=time(3), cache_mode='remake')

,day,featurizer
id,,
2,2,featurizer_A
3,2,featurizer_A
4,2,featurizer_A
5,2,featurizer_A
6,2,featurizer_A


In [11]:
dataset.read(from_timestamp=time(1), to_timestamp=time(5), cache_mode='remake')

,day,featurizer
id,,
4,4,featurizer_A
5,4,featurizer_A
6,4,featurizer_A
7,4,featurizer_A
8,4,featurizer_A


Between the days 1 and 5, two revisions took place: (2) and (4). But, since the revision (4) is major, it voids the output of the revision (2).

We encourage you to play with this code, downloading and reading data for different timeframes, and comparing with `summary_df` table.

`UpdatableDataset` supports `count`, `selector` and `columns` arguments for reading, as well as `as_data_frame_source` method that converts this dataset to the `DataFrameSource` interface. The guidelines of dataset exploration are the same as for `Dataset`.

## Updatable Featurization Job

`UpdableFeaturizationJob` is an analogue to `FeaturizationJob` that supports updates. The arguments of this class are essentially the same as for `FeaturizationJob`, because under the hood `UpdatableFeaturizationJob` only sorts out the timeframes, and spawns `FeaturizationJob` instances for actual featurization.

The only difference lies with `DataSource` that the job consumes. `UpdatableFeaturizationJob` requires you to provide a full data source that is used in major updates. If you want to have minor updates, you must also provide a factory that created a `DataSource` for a given timeframe.

Let's create an `UpdatableFeaturizationJob` for the Titanic dataset. Since this dataset doesn't actually have the time dependency, we will just split it into 3 groups, by `Embarked` field:
* for the initial major revision, `Embarked` will be set to None for all passengers (as if made before the trip started)
* following revisions will add the `Embarked` fields

The `Embarked` field is distributed as follows:

In [12]:
df = pd.read_csv('titanic.csv')
df.groupby(df.Embarked.fillna("NONE")).size()

Embarked
C       168
NONE      2
Q        77
S       644
dtype: int64

The data sources are implemented as follows:

In [13]:
from tg.common.datasets.access import DataSource

class EmbarkedDataSource(DataSource):
    def __init__(self, embarked):
        self.embarked = embarked
        
    def get_data(self):
        df = pd.read_csv('titanic.csv')
        if self.embarked is None:
            df.Embarked = 'NONE'
        else:
            df = df.loc[df.Embarked == self.embarked]
        return Query.df(df)
        
def source_factory(from_timestamp, to_timestamp):
    if to_timestamp.day == 3:
        return EmbarkedDataSource('C')
    elif to_timestamp.day == 5:
        return EmbarkedDataSource('Q')
    else:
        return EmbarkedDataSource('S')

Now we will implement and run `UpdatableFeaturizationJob`. Note that we must take some action regarding the dataframe indices: before we were simply ignoring it, but now we have to make sure that index is a unique identifier for an entity, i.e., passenger.

In [14]:
from tg.common.datasets.featurization import UpdatableFeaturizationJob, DataframeFeaturizer

dataset_buffer = MemoryFileSyncer()

class PassengerFeaturizer(DataframeFeaturizer):
    def __init__(self):
        super(PassengerFeaturizer, self).__init__(row_selector=lambda z: z)
    
    def _postprocess(self, df):
        return df.set_index('PassengerId')
    
job = UpdatableFeaturizationJob(
    name = 'test_featurization_job',
    version = 'v1',
    full_data_source=EmbarkedDataSource(None),
    update_data_source_factory=source_factory,
    featurizers = dict(passengers = PassengerFeaturizer()),
    syncer = dataset_buffer,
    limit = None,
    reporting_frequency=None
)

for i in [0,2,4,6]:
    job.run(current_time = time(i),custom_revision_id=str(i))

2022-12-28 14:20:20.265386 INFO: Starting lesvik job test_featurization_job, version v1


2022-12-28 14:20:20.266654 INFO: Additional settings limit NONE, reporting NONE


2022-12-28 14:20:20.267648 INFO: 0 previous revisions are found


2022-12-28 14:20:20.268154 INFO: Running with id 0 at 2020-01-01 00:00:00, revision is MAJOR


2022-12-28 14:20:20.268623 INFO: Featurization Job test_featurization_job at version v1 has started


...

2022-12-28 14:20:20.482652 INFO: Uploading data


2022-12-28 14:20:20.483651 INFO: Featurization job completed


2022-12-28 14:20:20.484254 INFO: 644 were processed


2022-12-28 14:20:20.484739 INFO: Uploading new description


2022-12-28 14:20:20.488453 INFO: Job finished


I have given a meaningful names to revisions with `custom_uid` argument. This is not necessary in general, as the order of revision is reflected in `description.parquet`.

Let's look at the revisions created. I will also add the information about records that were processed.

In [15]:
from collections import OrderedDict



def get_embarkation_by_revision(buffer):
    desc_df = pd.read_parquet(buffer.get_file_stream('description.parquet'))
    rows = []
    for key in buffer.cache:
        if key == 'description.parquet':
            continue
        df = buffer.get_parquet(key)
        df = df.groupby('Embarked').size().to_frame().transpose().iloc[0]
        row = OrderedDict()
        row['partition'] = key.split('/')[0]
        row['file'] = key.split('/')[2]
        for s in Query.series(df):
            row[s.key] = s.value
        
        rows.append(row)
        
    edf = pd.DataFrame(rows)
    desc_df = desc_df.merge(edf.set_index('partition'), left_on='name',right_index=True)
    return desc_df

desc_df = get_embarkation_by_revision(dataset_buffer)
desc_df

,name,timestamp,is_major,version,file,NONE,C,Q,S
0,0,2020-01-01,True,v1,9cee2b45-6757-4d05-a965-23bbe7156beb.parquet,891.0,NaN,NaN,NaN
1,2,2020-01-03,False,v1,8877c73f-43db-49d6-8436-0088ffdc6a8b.parquet,NaN,168.0,NaN,NaN
2,4,2020-01-05,False,v1,2d6d2392-9761-4822-b67e-53e00b81d60c.parquet,NaN,NaN,77.0,NaN
3,6,2020-01-07,False,v1,032f775d-7f9f-4b40-9bb7-2cd72519d590.parquet,NaN,NaN,NaN,644.0


As we see, all worked as expected.

The created dataset can be then accessed via `UpdatableDataset` class.

In [16]:
dataset = UpdatableDataset(
    location = './temp/updatable_dataset_2',
    featurizer_name = 'passengers',
    syncer = dataset_buffer
)
df = dataset.read(cache_mode='remake', partition_name_column='partition_name')
df = df.merge(
    dataset.get_desription_as_df().set_index('name'),
    left_on='partition_name',
    right_index=True)
df.groupby(['Embarked','timestamp']).size().to_frame('records_count').reset_index()

,Embarked,timestamp,records_count
0,C,2020-01-03,168
1,NONE,2020-01-01,2
2,Q,2020-01-05,77
3,S,2020-01-07,644


The handy argument `partition_name_column` adds to each rows the name of the revision that has produced this row. This name may be then used to merge with the description for further technical information.

## `UpdatableDatasetScoringJob`

There are probably lots of scenarios how the UpdatableDatasets may be processed. So far, we have suppored one scenario, `UpdatableDatasetScoringJob`. This job is mostly designed to compute scores for entities in datasets for different purposes. So the actions performed are:

* Get the last time when the job was run
* Download updates from dataset(s) from this last time
* Compute scores for the update
* Upload the updates for the dataset

Such job consists of UpdatableDatasetScoringMethods, each method knows:
* The dataset it pulls data from
* The function that must be applied to the resulting dataframe. The function must preserve the dataframe index.

Let's define such scoring method and job.

In [17]:
from tg.common.datasets.featurization import UpdatableDatasetScoringJob, UpdatableDatasetScoringMethod
from datetime import timedelta

def compute_scores(df):
    return df[['Survived','Embarked']]

scores_buffer = MemoryFileSyncer()

job = UpdatableDatasetScoringJob(
    name = 'scoring',
    version = '',
    dst_syncer = scores_buffer,
    methods = [
        UpdatableDatasetScoringMethod(
            'passenger_scores',
            dataset_buffer,
            'passengers',
            compute_scores        
        )
    ]
)

for i in [0,2,4,6]:
    job.run(current_time = time(i)+timedelta(hours=1), custom_revision_id=str(i))

The addition of one hour is required to reflect the fact that the scoring job would need to run _after_ the initial job. The 

In [18]:
get_embarkation_by_revision(scores_buffer)

,name,timestamp,is_major,version,file,NONE,C,Q,S
0,0,2020-01-01 01:00:00,True,,e94bf29e-9cc4-42ef-8f06-0315a822d532.parquet,891.0,NaN,NaN,NaN
1,2,2020-01-03 01:00:00,False,,2bd928b5-f354-4958-9410-746774ea2d0e.parquet,NaN,168.0,NaN,NaN
2,4,2020-01-05 01:00:00,False,,20ebdf31-5b8b-4f4d-a585-7f7eb73c5807.parquet,NaN,NaN,77.0,NaN
3,6,2020-01-07 01:00:00,False,,29a15853-1fe2-49e6-bcbc-7a28bf4573ed.parquet,NaN,NaN,NaN,644.0


Again, as we see, all worked as expected: each scoring job instance have only processed the data from the revision that was unprocessed at the time.

Let's change this behavior for demonstration purposes. With the `custom_start_time` argument, I will force the job to obtain all the changes made in the initial dataset since the time 0, not since that last run of the job.

In [19]:
scores_buffer = MemoryFileSyncer()

job = UpdatableDatasetScoringJob(
    name = 'scoring',
    version = '',
    dst_syncer = scores_buffer,
    methods = [
        UpdatableDatasetScoringMethod(
            'passenger_scores',
            dataset_buffer,
            'passengers',
            compute_scores        
        )
    ]
)

for i in [0,2,4,6]:
    job.run(
        current_time = time(i)+timedelta(hours=1), 
        custom_revision_id=str(i),
        custom_start_time=time(0)
    )

In [20]:
get_embarkation_by_revision(scores_buffer)

,name,timestamp,is_major,version,file,NONE,C,Q,S
0,0,2020-01-01 01:00:00,True,,e7badd8a-e8ee-4a7d-9521-f5345fbc2c34.parquet,891.0,NaN,NaN,NaN
1,2,2020-01-03 01:00:00,True,,fb9582ff-1e13-4389-b718-696bc88c756b.parquet,723.0,NaN,NaN,NaN
1,2,2020-01-03 01:00:00,True,,5587e92b-9d71-4f5e-9502-30737a6f2cdb.parquet,NaN,168.0,NaN,NaN
2,4,2020-01-05 01:00:00,True,,325cbab0-1d3e-44ae-86b5-8973677d50b2.parquet,NaN,NaN,77.0,NaN
2,4,2020-01-05 01:00:00,True,,bdeb970a-22d3-4036-a0ea-0ae319ef080b.parquet,NaN,168.0,NaN,NaN
2,4,2020-01-05 01:00:00,True,,09fb89bd-70cb-4e06-9b89-d7c2ef982857.parquet,646.0,NaN,NaN,NaN
3,6,2020-01-07 01:00:00,True,,c29aa577-79c7-4d1c-9442-89ecf3cac2f6.parquet,NaN,NaN,NaN,644.0
3,6,2020-01-07 01:00:00,True,,db3d8fa6-1cb9-4de8-a2a5-815b9ae804fd.parquet,2.0,NaN,NaN,NaN
3,6,2020-01-07 01:00:00,True,,4c6683a0-ff24-4b17-aff3-5051d66529f4.parquet,NaN,NaN,77.0,NaN
3,6,2020-01-07 01:00:00,True,,5fc276af-15c0-4344-a4c7-a166329483d8.parquet,NaN,168.0,NaN,NaN


Why are there so many files? Why are the different embarkations always in different files?

Because when we read changes from the revisions 0, 2, and 4, there are 3 physical files to read (1 at each revisions). We don't know for sure how these files are organized: maybe they override records of the previous ones, but _maybe_ they just add records, and _maybe_ if we try to read them all at once, we will overload the memory. This is why instead of `read` method,  `read_iter` is used. For the (4) revision, it yields: 

* first the dataframe from the revision (4), 
* then the dataframe from the revision (2), minus records in revision (4)
* then the dataframe from the revision (0) minus records in revision (2), (0)

If the data would be partitioned, it would further increase amount of dataframes.

Each of these dataframes has an important property: it is not bigger that something, created by `FeaturizationJob`. Therefore, it will not overload the memory. So regardless of how long this system is run untouched, the data accumulation will not result in memory overflow. This approach should be applied to all other jobs that process `UpdatableDatasets`


